In [1]:
%run -i 'cosim_import.py'

1.7.2.post284.dev0+g432288ce
2.9.0


In [2]:
caseH = 18
%run -i 'cosim_setup_andes.py'

EVA: ts=18[H], 50000 EVs, Total Q=360.34 MWh
Online 14481, Q=360.34 MWh, SoC=0.7678
Power(MW): Pt=-39.9433, Pc=-39.9433, Pd=-0.0
Ctrl: 0=7718; 1=6703; 
EVA: Load A from Aest.csv.
Working directory: "/home/jwang175/andes/jwang/notes"
> Loaded config from file "/home/jwang175/.andes/andes.rc"
> Loaded generated Python code in "/home/jwang175/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 6 processes.


Saved generated pycode to "/home/jwang175/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.3037 seconds.
Parsing input file "/home/jwang175/andes/jwang/case/ieee39_ev2.xlsx"...
Input file parsed in 0.1432 seconds.
ACEc <1> added BusFreq <BusFreq_11> linked to bus <1>
System internal structure set up in 0.0545 seconds.


In [3]:
%run -i 'cosim_loadsyn.py'

ANDES total load is: 58.564 p.u.


In [4]:
from jams import rted3

In [16]:
# --- set up EV generator data ---
ev_idx = 'PV_10'
ssa.PV.set(src='p0', idx=ev_idx, attr='v', value=sse.Ptc/ssa.config.mva)
ssa.PV.set(src='pmax', idx=ev_idx, attr='v', value=sse.Pu/ssa.config.mva)
ssa.PV.set(src='pmin', idx=ev_idx, attr='v', value=sse.Pl/ssa.config.mva)

# --- setup pandapower: ssp ---
ssp = to_pandapower(ssa)

# set EV generator as uncontrollable
ssp.gen.controllable.iloc[9] = False

# add gen cost, unit: $/MWh; G1-11. G10 EV, G11 Slack; 
# set EV as -10, for the cost of SFR mileage
c1 = [20, 20, 20, 20, 20, 20, 20, 20, 20, -10, 20]
c0 = [500, 380, 42, 380, 295, 400, 350, 330, 490, 0000, 550]
c2 = [0.014, 0.020, 0.194, 0.020, 0.0255, 0.0210, 0.230, 0.0222, 0.0150, 0.000, 0.0300]
gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 4] = np.array(c2) / ssa.config.mva
gen_cost[:, 5] = np.array(c1) / ssa.config.mva
gen_cost[:, 6] = np.array(c0) / ssa.config.mva

add_gencost(ssp, gen_cost)

# --- setup RTED: ssd ---
ssd = rted3(name='RTED', OutputFlag=0)
ssd.from_andes(ssa)
ssd.def_sfr(100, 100)
ssd.build()

# set EV generator as uncontrollable
ssd.gen.ctrl.loc[ev_idx] = 0

# set EV geenrator as type2
prumax = sse.g_frc()[0]
prdmax = sse.g_frc()[1]
ssd.def_type2([ev_idx], [prumax], [prdmax])

# Case data comes from a MPCE
# https://ieeexplore.ieee.org/document/9018441
# set ramp_5
ramp_15 = [156, 120, 130, 110, 80, 105, 90, 90, 150, 999, 200]
# ramp_hour = [80, 80, 80, 50, 50, 50, 30, 30, 30, 999, 30]
ssd.gen['ramp_5'] = np.array(ramp_15) / 3 / ssd.mva

# set cost
ssd.cost['c1'] = c1
ssd.cost['c2'] = c2
ssd.cost['c0'] = c0

# adjust SFR cost of EV lower than SynGen
ssd.cost['cru'] = [0] * ssd.gen.shape[0]
ssd.cost['cru'].loc[ev_idx] = - 0.000001
ssd.cost['crd'] = ssd.cost.cru

# --- benchmark ssd with ssp using DCOPF ---
pp.rundcopp(ssp)
dc_comp = ssp.res_gen.copy()
gb_res = ssd.solve(disable_sfr=True, disable_ramp=True, info=False)
dc_comp['p_mw(GB)'] = ssd.mva * gb_res['pg'].values
print(f"pp cost={ssp.res_cost}, gb cost={ssd.res_cost}")


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0112 seconds.
0: |F(x)| = 7.561249598
1: |F(x)| = 2.917108301
2: |F(x)| = 0.1313043343
3: |F(x)| = 0.0003572409148
4: |F(x)| = 4.286928146e-09
Converged in 5 iterations in 0.0145 seconds.
Power flow results are consistent. Conversion is successful.
-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0070 seconds.
0: |F(x)| = 7.561249598
1: |F(x)| = 2.917108301
2: |F(x)| = 0.1313043343
3: |F(x)| = 0.0003

pp cost=2224.8754288341015, gb cost=204910.30742817727
